In [44]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

In [2]:
def equation_plane(v0, v1, v2):
    point_mat = np.array([v0, v1, v2])
    abc = np.matmul(np.linalg.inv(point_mat), np.array([[1], [1], [1]]))
    p = np.concatenate([abc.T, np.array(-1).reshape(1, 1)], axis=1) / (np.sum(abc ** 2) ** 0.5)
    p = p.reshape(4)
    
    return p

In [196]:
np.linalg.inv(np.array([vertices[0], vertices[1], vertices[2]]))

array([[-0.0109789 , -0.16954258,  0.17999909],
       [-0.52340814,  0.2892106 ,  0.23517165],
       [ 0.20746798, -0.08383146, -0.14434425]])

In [198]:
a, b, c, d = equation_plane(vertices[0], vertices[1], vertices[2])
a ** 2 + b ** 2 + c ** 2

1.0

In [3]:
# calculate K
def calculate_K(vertices, triangles):
    K_list = []

    for i in range(len(triangles)):
        t = triangles[i]
        v0, v1, v2 = vertices[t[0]], vertices[t[1]], vertices[t[2]]
        p = equation_plane(v0, v1, v2)
        K = np.outer(p, p)
        K_list.append(K)
                
    return K_list

In [4]:
# create dict {v_id: [t_id0, t_id1, ...]}
def create_v_tr(triangles):
    v_tr = {}

    for i in range(len(triangles)):
        t = triangles[i]
        for v in t:
            if v in v_tr:
                v_tr[v].append(i)
            else:
                v_tr[v] = [i]
                
    return v_tr

In [5]:
def calculate_Q(K_list, v_tr):
    v_Q = {}

    for v_id, tr_ids in v_tr.items():
        Q = sum([K_list[el] for el in tr_ids])
        v_Q[v_id] = Q
        
    return v_Q

In [6]:
def find_edges(triangles):
    edge_1 = triangles[:, 0:2]
    edge_2 = triangles[:, 1:]
    edge_3 = np.concatenate([triangles[:, :1], triangles[:, -1:]], axis=1)
    edges = np.concatenate([edge_1, edge_2, edge_3], axis=0)
    
    unique_edges_trans, unique_edges_locs = np.unique(edges[:, 0] * (10 ** 10) + edges[:, 1], return_index=True)
    edges = edges[unique_edges_locs, :]
    
    return edges

In [7]:
def select_valid_pairs(vertices, triangles, t=0.001):
    
    valid_pairs = np.array([])
    
    for i in tqdm(range(len(vertices))):
        v = vertices[i]
        distances = np.sum((vertices - v) ** 2, axis=1) ** 0.5
        valid = np.where(distances < t)[0]
        pairs = np.vstack((np.ones(len(valid)) * i, valid)).T
        
        if i == 0:
            valid_pairs = pairs
        else:
            valid_pairs = np.vstack((valid_pairs, pairs))

    valid_pairs = valid_pairs[valid_pairs[:, 0] != valid_pairs[:, 1]]
    valid_pairs = valid_pairs.astype(int)
    
    edges = find_edges(triangles)
    valid_pairs = np.vstack((valid_pairs, edges))
    
    valid_pairs = valid_pairs[valid_pairs[:, 0] != valid_pairs[:, 1]]

    unique_valid_pairs_trans, unique_valid_pairs_loc = np.unique(valid_pairs[:, 0] * (10 ** 10) + valid_pairs[:, 1], return_index=True)
    valid_pairs = valid_pairs[unique_valid_pairs_loc, :]
    
    return valid_pairs

In [8]:
def select_valid_pairs3(vertices, triangles, t=0.001):
        
    edges = find_edges(triangles)
    valid_pairs = edges
    
    valid_pairs = valid_pairs[valid_pairs[:, 0] != valid_pairs[:, 1]]

    unique_valid_pairs_trans, unique_valid_pairs_loc = np.unique(valid_pairs[:, 0] * (10 ** 10) + valid_pairs[:, 1], return_index=True)
    valid_pairs = valid_pairs[unique_valid_pairs_loc, :]
    
    return valid_pairs

In [185]:
valid_pairs = select_valid_pairs(vertices, triangles, t=1)

100%|██████████| 2002/2002 [00:00<00:00, 16112.62it/s]


In [176]:
find_edges(triangles).shape

(9722, 2)

In [186]:
valid_pairs.shape

(9761, 2)

In [161]:
t = 1

for i in tqdm(range(1, len(vertices))):
    v = vertices[i]
    distances = np.sum((vertices - v) ** 2, axis=1) ** 0.5
    valid = np.where(distances < t)[0]
    pairs = np.vstack((np.ones(len(valid)) * i, valid)).T
    break

  0%|          | 0/2001 [00:00<?, ?it/s]


In [162]:
pairs

array([[1., 1.]])

In [150]:
valid_pairs.shape

(136, 2)

In [9]:
def select_valid_pairs2(vertices, v_tr, t=0.001):
    
    valid_pairs = []
    
    for i in tqdm(range(len(vertices))):
        v = vertices[i]
        distances = np.sum((vertices - v) ** 2, axis=1) ** 0.5
        valid = np.where(distances < t)[0]
        
        for el in valid:
            valid_pairs.append([i, el])
    
    valid_pairs = np.array(valid_pairs)
    valid_pairs = valid_pairs[valid_pairs[:, 0] != valid_pairs[:, 1]]
    valid_pairs = valid_pairs.astype(int)
    
    return valid_pairs

In [139]:
vertices.shape

(12000, 3)

In [135]:
valid_pairs = select_valid_pairs2(vertices, v_tr, t=1)

100%|██████████| 12000/12000 [00:02<00:00, 4318.80it/s]


In [10]:
def calculate_error(valid_pairs, v_Q, vertices):
    errors = []
    
    b = np.ones((vertices.shape[0], 1))
    vertices_4d = np.hstack((vertices, b))

    for i in range(len(valid_pairs)):
        pair = valid_pairs[i]
        v = (vertices_4d[pair[0]] - vertices_4d[pair[1]]) / 2
        
        try:
            Q1 = v_Q[pair[0]]
            Q2 = v_Q[pair[1]]
            Q = Q1 + Q2
        
            error = np.dot(np.dot(v, Q), v.T)    
        except KeyError:
            error = 10 ** 10
        
        errors.append(error)
        
    return errors

In [11]:
def create_queue(valid_pairs, errors):
    queue = np.hstack((valid_pairs, np.reshape(errors, (-1, 1))))
    # queue = queue[queue[:, 2] != 10 ** 10]
    queue = queue[np.argsort(queue[:, 2])]
    
    return queue

In [213]:
queue

array([[9.02000000e+02, 8.97000000e+02, 1.93389357e-02],
       [8.97000000e+02, 9.02000000e+02, 1.93389357e-02],
       [1.33200000e+03, 1.39700000e+03, 2.18379684e-02],
       ...,
       [1.78200000e+03, 1.81500000e+03, 1.31674490e+02],
       [1.10000000e+01, 5.10000000e+01, 1.87732560e+02],
       [5.10000000e+01, 1.10000000e+01, 1.87732560e+02]])

In [12]:
def remove_vertices(queue, vertices, triangles):
    # v1_deleted = []
    
    # to remove vertice
    for q in tqdm(queue):
        v0, v1, error = q
        v0, v1 = int(v0), int(v1)
        
        # update v_tr
        v_tr = create_v_tr(triangles)

        # if v0 not in v1_deleted and v1 not in v1_deleted:
        # 1. update v1 values in vertices
        vertices[v0] = (vertices[v0] + vertices[v1]) / 2

        # 2. change v2 to v1 in planes
        try:
            for tr in v_tr[v1]:
                triangles[tr][triangles[tr] == v1] = v0
        except KeyError:
            pass
        
        # 3. update cost for v1/v2
        pass

        # 4. remove v2 from cost
        # v1_deleted.append(v1)

        # 5. remove v2 from vertices
        pass
    
    return vertices, triangles

In [17]:
def remove_one_vertice(v0, v1, vertices, triangles, v_tr):
    vertices[v0] = (vertices[v0] + vertices[v1]) / 2

    # 2. change v2 to v1 in planes
    try:
        for tr in v_tr[v1]:
            triangles[tr][triangles[tr] == v1] = v0
    except KeyError:
        pass

    return vertices, triangles

In [586]:
valid_pairs[~((valid_pairs[:, 0] == 564) | (valid_pairs[:, 1] == 564))].shape

(622, 2)

In [587]:
valid_pairs.shape

(637, 2)

In [14]:
def remove_vertices2(vertices, triangles, ratio=0.1):
    init_triangles = len(triangles)
    valid_pairs = select_valid_pairs(vertices, triangles, t=1)
        
    while len(triangles) > init_triangles * ratio:
        K_list = calculate_K(vertices, triangles)
        v_tr = create_v_tr(triangles)
        v_Q = calculate_Q(K_list, v_tr)

        errors = calculate_error(valid_pairs, v_Q, vertices)
        queue = create_queue(valid_pairs, errors)

        v0, v1, error = queue[0]
        v0, v1 = int(v0), int(v1)

        vertices, triangles = remove_one_vertice(v0, v1, vertices, triangles)
        valid_pairs = valid_pairs[~((valid_pairs[:, 0] == v1) | (valid_pairs[:, 1] == v1))]

        triangles = np.unique(triangles, axis=0)
        triangles = triangles[(triangles[:, 0] != triangles[:, 1]) & (triangles[:, 1] != triangles[:, 2]) & (triangles[:, 2] != triangles[:, 0])]

        print(len(triangles))
        
    return vertices, triangles

In [616]:
mesh4 = o3d.geometry.TriangleMesh.create_torus()
print(mesh4)

vertices = np.asarray(mesh4.vertices)
triangles = np.asarray(mesh4.triangles)

vertices, triangles = remove_vertices2(vertices, triangles, ratio=0.1)

mesh5 = o3d.geometry.TriangleMesh()
mesh5.vertices = o3d.utility.Vector3dVector(vertices)
mesh5.triangles = o3d.utility.Vector3iVector(triangles)

mesh5.remove_unreferenced_vertices()

print(mesh5)

o3d.io.write_triangle_mesh('torus8.obj', mesh5)

TriangleMesh with 600 points and 1200 triangles.


100%|██████████| 600/600 [00:00<00:00, 4081.32it/s]


1200
1200
1200
1200
1200
1199
1199
1199
1199


KeyboardInterrupt: 

In [597]:
mesh4 = o3d.geometry.TriangleMesh.create_torus()
print(mesh4)

vertices = np.asarray(mesh4.vertices)
triangles = np.asarray(mesh4.triangles)

TriangleMesh with 600 points and 1200 triangles.


In [43]:
errors

NameError: name 'errors' is not defined

In [164]:
def update_queue(queue, v0, v1, v_Q, vertices):
    # drop invalid pairs from queue for v1
    queue = queue[~((queue[:, 0] == v1) | (queue[:, 1] == v1))]

    # update error value for v0
    value_pairs_to_update = queue[(queue[:, 0] == v0) | (queue[:, 1] == v0)][:, :2].astype(int)
    errors_to_update = np.array(calculate_error(value_pairs_to_update, v_Q, vertices))

    rows = np.argwhere((queue[:, 0] == v0) | (queue[:, 1] == v0))    
    queue[rows, 2] = np.reshape(errors_to_update, (-1, 1))
    
    # sort again
    queue = queue[np.argsort(queue[:, 2])]
    
    return queue

In [165]:
def remove_vertices3(vertices, triangles, ratio, t):

    valid_pairs = select_valid_pairs(vertices, triangles, t=t)
    num_steps = int(len(triangles) * (1 - ratio) / 2)
    K_list = calculate_K(vertices, triangles)
    v_tr = create_v_tr(triangles)
    v_Q = calculate_Q(K_list, v_tr)
    errors = calculate_error(valid_pairs, v_Q, vertices)
    queue = create_queue(valid_pairs, errors)
    
    for i in tqdm(range(num_steps)):
        
        v0, v1, error = queue[0]
        v0, v1 = int(v0), int(v1)
        v_Q[v0] = v_Q[v0] + v_Q[v1]
        del v_Q[v1]
        
        vertices, triangles = remove_one_vertice(v0, v1, vertices, triangles, v_tr)
        valid_pairs = valid_pairs[~((valid_pairs[:, 0] == v1) | (valid_pairs[:, 1] == v1))]

        triangles = np.unique(triangles, axis=0)
        triangles = triangles[(triangles[:, 0] != triangles[:, 1]) & (triangles[:, 1] != triangles[:, 2]) & (triangles[:, 2] != triangles[:, 0])]

        v_tr = create_v_tr(triangles)
        queue = update_queue(queue, v0, v1, v_Q, vertices)
        
        if len(queue) == 0:
            break

    return vertices, triangles

In [168]:
mesh4 = o3d.geometry.TriangleMesh.create_torus()
print(mesh4)

torus_v = np.asarray(mesh4.vertices)
torus_tr = np.asarray(mesh4.triangles)

torus_v2, torus_tr2 = remove_vertices3(torus_v, torus_tr, ratio=0.1, t=1)

TriangleMesh with 600 points and 1200 triangles.


100%|██████████| 540/540 [00:02<00:00, 232.78it/s]


In [169]:
mesh5 = o3d.geometry.TriangleMesh()
mesh5.vertices = o3d.utility.Vector3dVector(torus_v2)
mesh5.triangles = o3d.utility.Vector3iVector(torus_tr2)

mesh5.remove_unreferenced_vertices()

print(mesh5)

o3d.io.write_triangle_mesh('torus11.obj', mesh5)

TriangleMesh with 60 points and 120 triangles.


True

In [138]:
mesh4 = o3d.geometry.TriangleMesh.create_torus()
print(mesh4)

vertices = np.asarray(mesh4.vertices)
triangles = np.asarray(mesh4.triangles)

valid_pairs = select_valid_pairs(vertices, triangles, t=1)
K_list = calculate_K(vertices, triangles)
v_tr = create_v_tr(triangles)
v_Q = calculate_Q(K_list, v_tr)

TriangleMesh with 600 points and 1200 triangles.


100%|██████████| 600/600 [00:00<00:00, 4999.98it/s]


In [139]:
t1 = time.time()
errors = calculate_error(valid_pairs, v_Q, vertices)
t2 = time.time()
print(t2 - t1)

queue = create_queue(valid_pairs, errors)
t3 = time.time()
print(t3 - t2)

v0, v1, error = queue[0]
v0, v1 = int(v0), int(v1)        
vertices, triangles = remove_one_vertice(v0, v1, vertices, triangles, v_tr)
valid_pairs = valid_pairs[~((valid_pairs[:, 0] == v1) | (valid_pairs[:, 1] == v1))]
t4 = time.time()
print(t4 - t3)

triangles = np.unique(triangles, axis=0)
triangles = triangles[(triangles[:, 0] != triangles[:, 1]) & (triangles[:, 1] != triangles[:, 2]) & (triangles[:, 2] != triangles[:, 0])]
t5 = time.time()
print(t5 - t4)

v_tr = create_v_tr(triangles)
t6 = time.time()
print(t6 - t5)

0.555889368057251
0.01034855842590332
0.0
0.0040514469146728516
0.0


In [140]:
v0, v1, _ = queue[0].astype(int)
v0, v1

(135, 155)

In [141]:
queue.shape

(86462, 3)

In [143]:
queue[:5]

array([[1.35000000e+02, 1.55000000e+02, 7.00966071e-05],
       [1.55000000e+02, 1.35000000e+02, 7.00966071e-05],
       [4.75000000e+02, 4.95000000e+02, 7.00966071e-05],
       [4.95000000e+02, 4.75000000e+02, 7.00966071e-05],
       [4.25000000e+02, 4.45000000e+02, 7.00966071e-05]])

In [144]:
queue[(queue[:, 0] == v0) | (queue[:, 1] == v0)][:5]

array([[1.35000000e+02, 1.55000000e+02, 7.00966071e-05],
       [1.55000000e+02, 1.35000000e+02, 7.00966071e-05],
       [1.15000000e+02, 1.35000000e+02, 7.00966071e-05],
       [1.35000000e+02, 1.15000000e+02, 7.00966071e-05],
       [1.35000000e+02, 1.75000000e+02, 6.82718614e-04]])

In [146]:
value_pairs_to_update[0], errors_to_update[0]

(array([115, 135]), 0.00018971884785696578)

In [147]:
queue[(queue[:, 0] == v0) | (queue[:, 1] == v0)][:5]

array([[1.15000000e+02, 1.35000000e+02, 1.89718848e-04],
       [1.35000000e+02, 1.15000000e+02, 1.89718848e-04],
       [1.35000000e+02, 1.75000000e+02, 3.92416720e-04],
       [1.75000000e+02, 1.35000000e+02, 3.92416720e-04],
       [9.50000000e+01, 1.35000000e+02, 1.12148461e-03]])

In [148]:
queue[:5]

array([[4.75000000e+02, 4.95000000e+02, 7.00966071e-05],
       [4.95000000e+02, 4.75000000e+02, 7.00966071e-05],
       [4.25000000e+02, 4.45000000e+02, 7.00966071e-05],
       [4.45000000e+02, 4.25000000e+02, 7.00966071e-05],
       [5.75000000e+02, 5.55000000e+02, 7.00966071e-05]])

In [63]:
def queue_to_dict(queue):
    queue_d = {}
    
    for el in queue:
        v0, v1, error = el
        v0, v1 = int(v0), int(v1)
        
        queue_d[str(v0) + '_' + str(v1)] = error
    
    return queue_d

In [64]:
queue_d = queue_to_dict(queue)

In [65]:
queue_d

{'135_155': 7.009660706283341e-05,
 '155_135': 7.009660706283341e-05,
 '475_495': 7.009660706283536e-05,
 '495_475': 7.009660706283536e-05,
 '425_445': 7.009660706283585e-05,
 '445_425': 7.009660706283585e-05,
 '575_555': 7.009660706283914e-05,
 '555_575': 7.009660706283914e-05,
 '495_515': 7.009660706283922e-05,
 '515_495': 7.009660706283922e-05,
 '435_415': 7.009660706283964e-05,
 '415_435': 7.009660706283964e-05,
 '175_155': 7.009660706284005e-05,
 '155_175': 7.009660706284005e-05,
 '75_95': 7.009660706284018e-05,
 '95_75': 7.009660706284018e-05,
 '45_25': 7.009660706284028e-05,
 '25_45': 7.009660706284028e-05,
 '215_235': 7.009660706284036e-05,
 '235_215': 7.009660706284036e-05,
 '565_545': 7.009660706284039e-05,
 '545_565': 7.009660706284039e-05,
 '265_245': 7.009660706284068e-05,
 '245_265': 7.009660706284068e-05,
 '375_395': 7.009660706284078e-05,
 '395_375': 7.009660706284078e-05,
 '75_55': 7.009660706284078e-05,
 '55_75': 7.009660706284078e-05,
 '345_325': 7.009660706284089e-0

In [549]:
queue

array([[1.55000000e+02, 1.35000000e+02, 7.00966071e-05],
       [4.95000000e+02, 4.75000000e+02, 7.00966071e-05],
       [4.45000000e+02, 4.25000000e+02, 7.00966071e-05],
       ...,
       [5.80000000e+02, 1.00000000e+00, 1.50550004e-02],
       [5.99000000e+02, 0.00000000e+00, 1.50550004e-02],
       [0.00000000e+00, 5.99000000e+02, 1.50550004e-02]])

In [535]:
valid_pairs = select_valid_pairs(vertices, triangles, t=0.00001)
valid_pairs

100%|██████████| 600/600 [00:00<00:00, 14999.57it/s]


array([[  5,  11],
       [  5,  25],
       [  5,  50],
       ...,
       [581,  24],
       [581, 499],
       [581, 564]])

In [112]:
dataset = o3d.data.BunnyMesh()
mesh = o3d.io.read_triangle_mesh(dataset.path)

In [251]:
mesh_path = 'dinosaur.obj'
mesh = o3d.io.read_triangle_mesh(mesh_path)

In [252]:
vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)

In [253]:
vertices[0]

array([-11.7397995 , -21.08160019, -48.98669815])

In [254]:
K_list = calculate_K(vertices, triangles)
v_tr = create_v_tr(triangles)
v_Q = calculate_Q(K_list, v_tr)

In [255]:
valid_pairs[valid_pairs[:, 0] == 0]

array([[ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 0,  5],
       [ 0, 10],
       [ 0, 16]])

In [256]:
valid_pairs[valid_pairs[:, 1] == 0]

array([[ 1,  0],
       [ 3,  0],
       [ 4,  0],
       [ 5,  0],
       [10,  0],
       [16,  0]])

In [257]:
valid_pairs = select_valid_pairs(vertices, triangles, t=1)
errors = calculate_error(valid_pairs, v_Q, vertices)
queue = create_queue(valid_pairs, errors)

100%|██████████| 2002/2002 [00:00<00:00, 14574.78it/s]


In [258]:
queue.shape

(9761, 3)

In [102]:
queue_short = queue[:10**5, :]

In [259]:
vertices, triangles = remove_vertices(queue, vertices, triangles, v_tr)

100%|██████████| 9761/9761 [00:00<00:00, 129272.09it/s]


In [260]:
triangles.shape

(4000, 3)

In [261]:
triangles = np.unique(triangles, axis=0)
triangles.shape

(3980, 3)

In [262]:
triangles

array([[   1,   60,   60],
       [   3,   35,   64],
       [   3,  136,   61],
       ...,
       [2000, 1972, 2000],
       [2000, 1978, 1969],
       [2000, 1986, 2000]], dtype=int32)

In [263]:
triangles = np.unique(triangles, axis=0)
triangles = triangles[(triangles[:, 0] != triangles[:, 1]) & (triangles[:, 1] != triangles[:, 2]) & (triangles[:, 2] != triangles[:, 0])]
triangles.shape

(2117, 3)

In [264]:
triangles

array([[   3,   35,   64],
       [   3,  136,   61],
       [   5,   24,   60],
       ...,
       [1998, 1996, 1988],
       [1999, 1923, 1985],
       [2000, 1978, 1969]], dtype=int32)

In [93]:
# reduce amount of vertices
# unique = np.unique(triangles)
# mask = np.isin(np.arange(len(vertices)), unique)
# vertices = vertices[mask]

In [265]:
mesh2 = o3d.geometry.TriangleMesh()
mesh2.vertices = o3d.utility.Vector3dVector(vertices)
mesh2.triangles = o3d.utility.Vector3iVector(triangljes)

In [266]:
o3d.io.write_triangle_mesh('dinosaur6.obj', mesh2)

True

In [ ]:
# remove_duplicated_vertices': <instancemethod remove_duplicated_vertices at 0x000002BF4739EBE0>,
# 'remove_duplicated_triangles': <instancemethod remove_duplicated_triangles at 0x000002BF4739EC40>,
# 'remove_unreferenced_vertices': <instancemethod remove_unreferenced_vertices at 0x000002BF4739ECA0>,
# 'remove_degenerate_triangles': <instancemethod remove_degenerate_triangles at 0x000002BF4739ED00>,
# 'remove_non_manifold_edges

In [273]:
o3d.geometry.TriangleMesh.__dict__

mappingproxy({'__init__': <instancemethod __init__ at 0x000002BF4739E970>,
              '__doc__': 'TriangleMesh class. Triangle mesh contains vertices and triangles represented by the indices to the vertices. Optionally, the mesh may also contain triangle normals, vertex normals and vertex colors.',
              '__module__': 'open3d.cpu.pybind.geometry',
              '__copy__': <instancemethod __copy__ at 0x000002BF4739E8E0>,
              '__deepcopy__': <instancemethod __deepcopy__ at 0x000002BF4739E940>,
              '__repr__': <instancemethod __repr__ at 0x000002BF4739E9A0>,
              '__add__': <instancemethod __add__ at 0x000002BF4739EA00>,
              '__iadd__': <instancemethod __iadd__ at 0x000002BF4739EA60>,
              'compute_triangle_normals': <instancemethod compute_triangle_normals at 0x000002BF4739EAC0>,
              'compute_vertex_normals': <instancemethod compute_vertex_normals at 0x000002BF4739EB20>,
              'compute_adjacency_list': <instanc

In [324]:
o3d.geometry.TriangleMesh.create_torus()

TriangleMesh with 600 points and 1200 triangles.

In [518]:
# mesh4 = o3d.geometry.TriangleMesh.create_octahedron()
mesh4 = o3d.geometry.TriangleMesh.create_torus()
print(mesh4)

vertices = np.asarray(mesh4.vertices)
triangles = np.asarray(mesh4.triangles)

TriangleMesh with 600 points and 1200 triangles.


In [482]:
K_list = calculate_K(vertices, triangles)
v_tr = create_v_tr(triangles)
v_Q = calculate_Q(K_list, v_tr)

valid_pairs = select_valid_pairs(vertices, triangles, t=0.001)
errors = calculate_error(valid_pairs, v_Q, vertices)
queue = create_queue(valid_pairs, errors)
# queue = queue[:5, :]

vertices, triangles = remove_vertices(queue[:4, :], vertices, triangles)

100%|██████████| 4/4 [00:00<00:00, 666.61it/s]


In [483]:
triangles[triangles[:, 0] == 515], triangles[triangles[:, 1] == 515], triangles[triangles[:, 2] == 515]

(array([[515, 516, 495],
        [515, 536, 516]], dtype=int32),
 array([[514, 515, 494],
        [494, 515, 495]], dtype=int32),
 array([[514, 535, 515],
        [535, 536, 515]], dtype=int32))

In [484]:
triangles[triangles[:, 0] == 495], triangles[triangles[:, 1] == 495], triangles[triangles[:, 2] == 495]

(array([[495, 476, 455],
        [495, 496, 495],
        [495, 496, 476],
        [495, 516, 496]], dtype=int32),
 array([[474, 495, 454],
        [454, 495, 455],
        [494, 495, 474],
        [474, 495, 495]], dtype=int32),
 array([[474, 495, 495],
        [495, 496, 495],
        [494, 515, 495],
        [515, 516, 495]], dtype=int32))

In [485]:
queue[0].astype(int)

array([155, 135,   0])

In [486]:
v_tr[135]

[228, 229, 230, 269, 270, 271]

In [487]:
v_tr = create_v_tr(triangles)

In [488]:
v_tr[135]

KeyError: 135

In [489]:
queue[4].astype(int)

array([515, 495,   0])

In [490]:
vertices, triangles = remove_vertices(queue[4:5, :], vertices, triangles)

100%|██████████| 1/1 [00:00<00:00, 986.66it/s]


In [491]:
triangles[triangles[:, 0] == 515], triangles[triangles[:, 1] == 515], triangles[triangles[:, 2] == 515]

(array([[515, 476, 455],
        [515, 496, 515],
        [515, 496, 476],
        [515, 516, 515],
        [515, 516, 496],
        [515, 536, 516]], dtype=int32),
 array([[474, 515, 454],
        [454, 515, 455],
        [494, 515, 474],
        [474, 515, 515],
        [514, 515, 494],
        [494, 515, 515]], dtype=int32),
 array([[474, 515, 515],
        [515, 496, 515],
        [494, 515, 515],
        [515, 516, 515],
        [514, 535, 515],
        [535, 536, 515]], dtype=int32))

In [492]:
triangles[triangles[:, 0] == 495], triangles[triangles[:, 1] == 495], triangles[triangles[:, 2] == 495]

(array([], shape=(0, 3), dtype=int32),
 array([], shape=(0, 3), dtype=int32),
 array([], shape=(0, 3), dtype=int32))

In [493]:
queue[5].astype(int)

array([435, 415,   0])

In [494]:
triangles = np.unique(triangles, axis=0)
triangles = triangles[(triangles[:, 0] != triangles[:, 1]) & (triangles[:, 1] != triangles[:, 2]) & (triangles[:, 2] != triangles[:, 0])]

In [495]:
triangles[triangles[:, 0] == 135], triangles[triangles[:, 1] == 135], triangles[triangles[:, 2] == 135]

(array([], shape=(0, 3), dtype=int32),
 array([], shape=(0, 3), dtype=int32),
 array([], shape=(0, 3), dtype=int32))

In [496]:
mesh5 = o3d.geometry.TriangleMesh()
mesh5.vertices = o3d.utility.Vector3dVector(vertices)
mesh5.triangles = o3d.utility.Vector3iVector(triangles)

In [497]:
mesh5.remove_unreferenced_vertices()

TriangleMesh with 595 points and 1190 triangles.

In [498]:
# queue 1 => TriangleMesh with 599 points and 1198 triangles.
# 2 => TriangleMesh with 598 points and 1196 triangles.
# 3 => TriangleMesh with 597 points and 1194 triangles.

In [499]:
o3d.io.write_triangle_mesh('torus5.obj', mesh5)

True

In [308]:
vertices, triangles

(array([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.],
        [-1.,  0.,  0.],
        [ 0., -1.,  0.],
        [ 0.,  0., -1.]]),
 array([[0, 0, 2],
        [0, 3, 2],
        [3, 4, 2],
        [4, 0, 2],
        [0, 5, 0],
        [0, 5, 3],
        [3, 5, 4],
        [4, 5, 0]], dtype=int32))

In [336]:
triangles

array([[  0,   1, 580],
       [  0,  21,   1],
       [  1,   2, 581],
       ...,
       [598, 599, 578],
       [599,   0, 580],
       [599, 580, 579]], dtype=int32)

In [314]:
np.asarray(mesh5.triangles)

array([[0, 0, 2],
       [0, 3, 2],
       [3, 4, 2],
       [4, 0, 2],
       [0, 5, 0],
       [0, 5, 3],
       [3, 5, 4],
       [4, 5, 0]], dtype=int32)

In [269]:
dataset = o3d.data.KnotMesh()
mesh3 = o3d.io.read_triangle_mesh(dataset.path)
o3d.io.write_triangle_mesh('knot.obj', mesh3)

[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/KnotMesh.ply
[Open3D WARNING] Failed to download from https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/KnotMesh.ply. Expection [Open3D Error] (class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > __cdecl open3d::utility::DownloadFromURL(const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &,const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &,const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &,const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &)) D:\a\Open3D\Open3D\cpp\open3d\utility\Download.cpp:173: Download failed with error code: Timeout was reached.
.


RuntimeError: [Open3D Error] (class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > __cdecl open3d::utility::DownloadFromURL(const class std::vector<class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> >,class std::allocator<class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > > > &,const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &,const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &,const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &)) D:\a\Open3D\Open3D\cpp\open3d\utility\Download.cpp:192: Downloading failed from available mirrors.


In [ ]:
mesh2 = o3d.geometry.TriangleMesh()
mesh2.vertices = o3d.utility.Vector3dVector([
    [1, 1, 1], # 0
    [0, 0, 0], # 1
    [0, 0, 1], # 2
    [0, 1, 0], # 3
    [1, 0, 0], # 4
    [0, 1, 1], # 5
    [1, 1, 0], # 6
    [1, 0, 1] # 7
])
mesh2.triangles = o3d.utility.Vector3iVector([
    [1, 4, 6],
    [1, 3, 6],
    [1, 2, 3],
    [3, 2, 5],
    []
])

In [526]:
# mesh4 = o3d.geometry.TriangleMesh.create_octahedron()
mesh4 = o3d.geometry.TriangleMesh.create_torus()
print(mesh4)

vertices = np.asarray(mesh4.vertices)
triangles = np.asarray(mesh4.triangles)

K_list = calculate_K(vertices, triangles)
v_tr = create_v_tr(triangles)
v_Q = calculate_Q(K_list, v_tr)

valid_pairs = select_valid_pairs(vertices, triangles, t=0.01)
errors = calculate_error(valid_pairs, v_Q, vertices)
queue = create_queue(valid_pairs, errors)

vertices, triangles = remove_vertices(queue, vertices, triangles)

triangles = np.unique(triangles, axis=0)
triangles = triangles[(triangles[:, 0] != triangles[:, 1]) & (triangles[:, 1] != triangles[:, 2]) & (triangles[:, 2] != triangles[:, 0])]

mesh5 = o3d.geometry.TriangleMesh()
mesh5.vertices = o3d.utility.Vector3dVector(vertices)
mesh5.triangles = o3d.utility.Vector3iVector(triangles)

mesh5.remove_unreferenced_vertices()

print(mesh5)

o3d.io.write_triangle_mesh('torus8.obj', mesh5)

TriangleMesh with 600 points and 1200 triangles.


100%|██████████| 2400/2400 [00:05<00:00, 473.85it/s]

TriangleMesh with 136 points and 349 triangles.


True

In [527]:
valid_pairs.shape

(2400, 2)

In [530]:
valid_pairs[:20, :]

array([[  0,   1],
       [  0,  21],
       [  0, 580],
       [  0, 599],
       [  1,   2],
       [  1,  22],
       [  1, 580],
       [  1, 581],
       [  2,   3],
       [  2,  23],
       [  2, 581],
       [  2, 582],
       [  3,   4],
       [  3,  24],
       [  3, 582],
       [  3, 583],
       [  4,   5],
       [  4,  25],
       [  4, 583],
       [  4, 584]])